In [21]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [22]:
print("download.default_directory=" + os.getcwd() + "\Downloads")

download.default_directory=C:\Users\marti\Desktop\Webscraping\Downloads


In [23]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.getcwd() + "\Downloads",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}
options.add_experimental_option('prefs', prefs)

In [24]:
start_page = "https://www.researchgate.net/search/authors?q"
test_name = "Michael Kabo"


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [33]:
def research_gate_scraper(record):

    profile = {
        "name" : "",
        "bio" : "",
        "dept" : "",
        "personal" : "",
        "academic" : "",
        "research" : "",
        "orcid" : "",
        "scopus" : ""
    }

    research_gate = "https://www.researchgate.net/search/authors?q"

    #Instatiate the webdriver and set to implicitly wait for page objects to load
    driver = webdriver.Chrome()
    driver.implicitly_wait(4) # seconds loading implicitly

    #Set driver to go to start_page and search a given name
    driver.get(research_gate)
    search_bar = driver.find_element_by_tag_name('input')
    search_bar.send_keys(record.faculty)
    search_bar.send_keys(Keys.ENTER)
    
    results = False
    if not driver.find_elements_by_xpath('//*[@id="rgw4_5ce5b6067a7fe"]/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div/div/div'):
        results = True
        

    #Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
    view_profile = driver.find_element_by_link_text('View Profile')
    view_profile.click()

    #Attempt to find the total number of research items for a given professsor 
    num_research_items = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit"]')
    num_research_items = num_research_items.text
    num_research_items = int(num_research_items)
    print(num_research_items)


    # grab the research interests
    interests = driver.find_elements_by_xpath('//*[@id="about"]/div/div/div[3]/div/div/div[2]/div/div[2]/div/div/div/div/div')
    interests = [item.text for item in interests]
    while '' in interests:
        interests.remove('')
    print(interests)

    # Retrieve all the urls of the first page
    research_a_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
    urls = [tag.get_attribute("href") for tag in research_a_tags]
    titles = [tag.text for tag in research_a_tags]
    paper_types = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
    paper_types = [item.text for item in paper_types]
    num_research_items -= 100

    current_url = driver.current_url

    page_count = 2
    while num_research_items > 0:
        driver.get(current_url + '/' + str(page_count))
        temp_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
        for tag in temp_tags:
            urls.append(tag.get_attribute("href"))
            titles.append(tag.text)
        temp_papers = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
        for paper in temp_papers:
            paper_types.append(paper.text)
        num_research_items -= 100
        page_count += 1

    while "Full-text available" in paper_types:
        paper_types.remove("Full-text available")

    driver.close()

    i = 0
    for url in urls:
        
        citation_text = ""
        title = ""
        abstract = ""
        book_title = ""
        journal_title = ""
        date = ""
        collabs = ""
        publisher = ""
        edition = ""
        series = ""
        volume = ""
        issue = ""
        chapter = ""
        pages = ""
        doi = ""
        issn = ""
        isbn = ""
        handle = ""

        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(4) # seconds loading implicitly

        #time.sleep(randint(5,10))

        driver.get(url)
        if results:
            if paper_types[i] == 'Article':

                # check for the DOI
                if driver.find_elements_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]'):
                    # Grab the DOI
                    meta = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]')
                    doi = meta.text.split(" ")
                    doi = doi[1]

                # check for the abstract
                if driver.find_elements_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]'):
                    # get the abstract
                    abstract = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]')    
                    abstract = abstract.text

                # then we'll get the collaborators
                collab = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/ul/li/div/div/div/div/div[2]/div/div/div/div/div/a')
                collabs = ""
                for person in collab:
                    collabs += person.text


                # Attempt to get the date
                date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
                date = date.text


                #download the citatation
                download_link = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/a')
                download_link.click()

                # click the text button
                text_radio_btn = driver.find_element_by_xpath("//span[contains(.,' Plain Text')]")
                text_radio_btn.click()

                # download the citation file
                download = driver.find_element_by_xpath('//button[@class="nova-c-button nova-c-button--align-center nova-c-button--radius-m nova-c-button--size-m nova-c-button--color-blue nova-c-button--theme-solid nova-c-button--width-auto"]')
                download.click()

                time.sleep(3)

                driver.close()


                # now that file is downloaded, need to open the file

                # here I grab the file name; the trick to this is making sure that its the only file in downloads
                citation_file = os.listdir(os.getcwd() + "\Downloads")[0]

                # grab the citation
                # i can probably get the name of the journal from the citation if thats something thats all over
                f = open("Downloads\\"+citation_file, "r")
                citation_text = f.read()
                f.close()
                # delete it
                os.remove("Downloads\\"+citation_file)

                citation_text


                pieces = citation_text.split('.')
                parts = []
                for piece in pieces:
                    parts.append(piece.strip())
                print(parts[3],",",parts[4])

                citations_file = open("research_gate_citations.txt", "a")

                citations_file.write(record.faculty.strip() + "|" + citation_text + "|" + titles[i] + "|" + paper_types[i] + "|")
                citations_file.write(abstract + "|" + book_title + "|" + journal_title + "|" + date + "|")
                citations_file.write(collabs + "|" + publisher + "|" + edition + "|" + series + "|")
                citations_file.write(volume + "|" + issue + "|" + chapter + "|" + pages)
                citations_file.write(doi + "|" + issn + "|" + isbn + "|" + handle + "|" + url + "\n")

            elif paper_types[i] == "Conference Paper":
                print("Some stuff")
            
        elif not results:
            no_results = open("research_gate_no_results.csv", "a")
            no_results.write(record.name.strip() + "," + record.college + "," + record.department + "," + record.url + "\n")
        i+=1


In [8]:
len(titles)

110

In [11]:
counter = 0
for url in urls:
    print(counter, ":", url)
    counter +=1
    
print(len(urls))

0 : https://www.researchgate.net/publication/315416439_Composite_mechanics_of_the_multilayer_structure_of_the_annulus_fibrosus
1 : https://www.researchgate.net/publication/313626183_Modeling_and_Analysis_of_Heart_Left_Ventricle_in_Diastole_Using_Finite_Element
2 : https://www.researchgate.net/publication/285611966_Machine_design_Redesigned
3 : https://www.researchgate.net/publication/261035675_Endoprosthetic_reconstruction_after_resection_of_musculoskeletal
4 : https://www.researchgate.net/publication/40768832_Cemented_Distal_Femoral_Endoprostheses_for_Musculoskeletal_Tumor_Improved_Survival_of_Modular_versus_Custom_Implants
5 : https://www.researchgate.net/publication/44571522_How_Long_Do_Endoprosthetic_Reconstructions_for_Proximal_Femoral_Tumors_Last
6 : https://www.researchgate.net/publication/44624077_Cemented_Endoprosthetic_Reconstruction_of_the_Proximal_Tibia_How_Long_Do_They_Last
7 : https://www.researchgate.net/publication/51381639_Hysteretic_pinching_of_human_secondary_osteons


Note that ResearchGate only loads 100 citations per page, if number of citations % 100 = 0 we go to second page

In [35]:
view_content = pd.read_csv("view_content.csv")
view_content.head()

,faculty,college,department,url
0,Sandor Samuels,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
1,Kurt Saunders,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
2,Hilary Silvia-Goldberg,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
3,Melanie Williams,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...
4,Gregory Young,College of Humanities,Department of Gender's & Women's Studies,https://www.csun.edu/humanities/gender-womens...


In [34]:
research_gate_scraper(view_content.iloc[0])

18
[]
Kay & Natschke, Mary , (2015)
The Holistic Worldview in Action: Evolution of Holistic Nurses Certification Programs , Journal of holistic nursing : official journal of the American Holistic Nurses' Association
Validation of Holistic Nursing Competencies: Role-Delineation Study, 2012 , Journal of holistic nursing : official journal of the American Holistic Nurses' Association
Holistic nursing: Becoming a catalyst for conscious change at end of life , Beginnings (American Holistic Nurses' Association)
Commentary on "Remembering Florence Nightingale's Panorama 21st-Century Nursing--At a Critical Crossroads" , Journal of holistic nursing : official journal of the American Holistic Nurses' Association
(2006) , Spirituality and care - Response
Exploring the Effects of Walking the Labyrinth , Journal of holistic nursing : official journal of the American Holistic Nurses' Association
Vanderpool, Harold & Dayao, Patricia , (2006)
Spirituality and Clinical Care: Exploring Developmental Cha